In [1]:
import numpy as np
from collections import Counter

In [2]:
import string
def preprocess(f_name):
    f=open(f_name, 'r')
    txt1=f.read().translate(str.maketrans("\t\r", "  "))
    txt1 = txt1.lower()
    "".join(txt1.split())
    txt=txt1.split('\n')
    sentence_corpora=[]
    sentence_labels=[]
    words=[]
    for i in range(0, 31984, 4):
        txt[i]=txt[i].lstrip('0123456789')
        txt[i]=txt[i].replace('\"','')
        txt[i]=txt[i].replace('.','')
        at=str(txt[i].strip())
        for elem in at.split(" "):
            words.append(elem.replace("<e1>","").replace("</e1>", "").replace("</e2>", "").replace("<e2>", ""))
        sentence_corpora.append(str(txt[i].strip().replace("<e1>","").replace("</e1>", "").replace("</e2>", "").replace("<e2>", "")))
        sentence_labels.append(str(txt[i+1].strip().replace("(e1,e2)", "").replace("(e2,e1)", "")))
    return sentence_corpora,sentence_labels,words


In [3]:
sentence_corpora,sentence_labels,words = preprocess("TRAIN_FILE.TXT")
print(type(sentence_corpora))
print(sentence_corpora[:10])

<class 'list'>
['the system as described above has its greatest application in an arrayed configuration of antenna elements', 'the child was carefully wrapped and bound into the cradle by means of a cord', 'the author of a keygen uses a disassembler to look at the raw assembly code', 'a misty ridge uprises from the surge', 'the student association is the voice of the undergraduate student population of the state university of new york at buffalohello sir', "this is the sprawling complex that is peru's largest producer of silver", 'the current view is that the chronic inflammation in the distal part of the stomach caused by helicobacter pylori infection results in an increased acid production from the non-infected upper corpus region of the stomach', 'people have been moving back into downtown', 'the lawsonite was contained in a platinum crucible and the counter-weight was a plastic crucible with metal pieces', 'the solute was placed inside a beaker and 5 ml of the solvent was pipetted 

In [4]:
#Setting Label values for Softmax Classifier
label_dict={"cause-effect": 0, 
            "instrument-agency": 1, 
            "product-producer": 2, 
            "content-container": 3, 
            "entity-origin": 4, 
            "entity-destination": 5, 
            "component-whole": 6,
            "member-collection": 7,
            "message-topic": 8,
            "other": 9}
final_labels=[]
for elem in sentence_labels:
    final_labels.append(label_dict[elem])
final_labels = np.array(final_labels)
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab,0)}
print(len(vocab_to_int))

22947


In [5]:
print(type(final_labels))
print(final_labels[:10])

<class 'numpy.ndarray'>
[6 9 1 9 7 9 0 5 3 5]


In [6]:
#Train Test Split
#from sklearn.model_selection import train_test_split
#x_train,x_test,y_train,y_test = train_test_split(sentence_corpora,final_labels,test_size=0.1,random_state=0)

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(sentence_corpora)
X_train_counts.shape

(7996, 19145)

In [8]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(7996, 19145)

In [9]:
print(type(X_train_tfidf))
print(X_train_tfidf[:5])

<class 'scipy.sparse.csr.csr_matrix'>
  (0, 17240)	0.051541530138
  (0, 16937)	0.227632272502
  (0, 1394)	0.150007682703
  (0, 5004)	0.28761472774
  (0, 484)	0.275084166443
  (0, 8043)	0.14980173258
  (0, 9252)	0.196402966198
  (0, 7738)	0.302849288377
  (0, 1222)	0.314926013731
  (0, 8728)	0.0900431528313
  (0, 1032)	0.142641658696
  (0, 1354)	0.390143029606
  (0, 3933)	0.344023927474
  (0, 11850)	0.0726551862258
  (0, 1131)	0.31857870051
  (0, 5855)	0.327002739085
  (1, 17240)	0.117531085922
  (1, 11850)	0.0828384694061
  (1, 3279)	0.313639706809
  (1, 18621)	0.135334659458
  (1, 2921)	0.359065750277
  (1, 19001)	0.367792245539
  (1, 1059)	0.0998160665467
  (1, 2393)	0.392242000488
  (1, 9108)	0.151053783916
  :	:
  (2, 1491)	0.176145183206
  (2, 13965)	0.345528048496
  (2, 1428)	0.331406521897
  (2, 3591)	0.323105319889
  (3, 17240)	0.067087255579
  (3, 11004)	0.507816222558
  (3, 14599)	0.507816222558
  (3, 18152)	0.469941926415
  (3, 7182)	0.151764530787
  (3, 16775)	0.48566117957

In [10]:
f=open('TEST_FILE_FULL.TXT', 'r')
txt1=f.read().translate(str.maketrans("\t\r", "  "))
txt1 = txt1.lower()
"".join(txt1.split())
txt=txt1.split('\n')
sentence_test=[]
labels_test=[]
words=[]
for i in range(0, 10866, 4):
    txt[i]=txt[i].lstrip('0123456789')
    txt[i]=txt[i].replace('\"','')
    txt[i]=txt[i].replace('.','')
    at=str(txt[i].strip())
    for elem in at.split(" "):
        words.append(elem.replace("<e1>","").replace("</e1>", "").replace("</e2>", "").replace("<e2>", ""))
    sentence_test.append(str(txt[i].strip().replace("<e1>","").replace("</e1>", "").replace("</e2>", "").replace("<e2>", "")))
    labels_test.append(str(txt[i+1].strip().replace("(e1,e2)", "").replace("(e2,e1)", "")))

In [11]:
final_labels_test=[]
for elem in labels_test:
    final_labels_test.append(label_dict[elem])
final_labels_test = np.array(final_labels_test)

In [12]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, final_labels)

In [37]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,2))),
                     ('tfidf', TfidfTransformer(use_idf=False)),
                     ('clf', MultinomialNB(alpha=0.01)),
])
text_clf = text_clf.fit(sentence_corpora, final_labels)

In [38]:
predicted = text_clf.predict(sentence_test)
np.mean(predicted == final_labels_test)

0.58667648141332351

In [13]:
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2)],
              'tfidf__use_idf': (True, False),
              'clf__alpha': (1e-2, 1e-3,1e-3),
              'vect__stop_words':('english',None)
}

In [14]:
gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)
gs_clf = gs_clf.fit(sentence_corpora, final_labels)

NameError: name 'text_clf' is not defined

In [36]:
gs_clf.best_score_
gs_clf.best_params_

{'clf__alpha': 0.01,
 'tfidf__use_idf': False,
 'vect__ngram_range': (1, 2),
 'vect__stop_words': None}

In [15]:
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
text_clf_svm = Pipeline([('vect', CountVectorizer(ngram_range=(1,2))),
                     ('tfidf', TfidfTransformer(use_idf=False)),
                     ('clf-svm', SGDClassifier(loss='squared_hinge', penalty='l2',
                                           alpha=2*1e-4, n_iter=800, random_state=100,learning_rate='constant',eta0=0.0009)),
 ])
_ = text_clf_svm.fit(sentence_corpora, final_labels)
predicted_svm = text_clf_svm.predict(sentence_test)
np.mean(predicted_svm == final_labels_test)

0.6205373573794627

In [23]:
f2 = open('new_file.txt','w')
pred = text_clf_svm.predict(['There is water in the bucket'])
for label, val in label_dict.items():
    if val == pred:
        print(label)
        f2.write(label)

content-container
